# 2015 - 02
This one is really a stretch to make use of a quantum circuit.
* Create a circuit with enough qubits for the binary representation of each of the 3 numbers, plus ancilla bits for the multiplication
* Find product by using controlled binary multiplication by using Toffoli gates

### Binary multiplaction  
Decimal: $2 \times 3 = 6$  
Binary:  $10 \times 11 = 110$  
Normal long multiplication, just multiply each bit of the first by each bit of the second and add up  
```bash
  10  
 x11  
----  
  10  
+100  
----  
=110  
```
This translates exactly to controlled-controlled-not gates targetting a $|0⟩$ qubit (Toffoli gataes)
* 0x0=0 -or- leave the target qubit as $|0⟩$
* 0x1=0 -or- leave the target qubit as $|0⟩$
* 1x0=0 -or- leave the target qubit as $|0⟩$
* 1x1=1 -or- flip the target qubit to $|1⟩$

CCX produces the same as the binary multiplication above as both controls must be a 1 for a result of 1 in the ancilla bit. Then we would need to sum it up and handle carry over
* Sum is an traditional XOR gate
* Carry over is a quantum CCX gate

But instead of building a binary multiplier, we can use a built in RGQFTMultiplier (Quantum Fourier Transform Multiplier) to do the multiplication for us.

In [ ]:
from IPython.display import display
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, AncillaRegister
from qiskit_aer import AerSimulator
from qiskit.compiler import transpile
from qiskit.circuit.library import RGQFTMultiplier

def compute_product(l: int, w: int, h: int):
  factor_size = max(l, w, h).bit_length()

  q_l = QuantumRegister(factor_size, name='l')
  q_w = QuantumRegister(factor_size, name='w')
  q_h = QuantumRegister(factor_size, name='h')
  q_lw = AncillaRegister(factor_size * 2, name='lw')
  q_lh = AncillaRegister(factor_size * 2, name='lh')
  q_wh = AncillaRegister(factor_size * 2, name='wh')
  c_lw = ClassicalRegister(factor_size * 2, name='c_lw')
  c_lh = ClassicalRegister(factor_size * 2, name='c_lh')
  c_wh = ClassicalRegister(factor_size * 2, name='c_wh')

  circuit = QuantumCircuit(q_l, q_w, q_h, q_lw, q_lh, q_wh, c_lw, c_lh, c_wh)

  def encode_dimension(value: int, reg: QuantumRegister):
    print(f"dimension: {format(value, f'0{factor_size}b')}")
    bin = f"{value:b}"
    for i, bit in enumerate(reversed(bin)): # reversed to start with LSB
      if bit == '1':
        circuit.x(reg[i]) # Apply x gate to flip bits to 1

  encode_dimension(l, q_l)
  encode_dimension(w, q_w)
  encode_dimension(h, q_h)
  circuit.barrier()

  mult = RGQFTMultiplier(factor_size)
  circuit.append(mult, q_l._bits + q_w._bits + q_lw._bits)
  circuit.append(mult, q_l._bits + q_h._bits + q_lh._bits)
  circuit.append(mult, q_w._bits + q_h._bits + q_wh._bits)
  circuit.barrier()

  circuit.measure(q_lw, c_lw)
  circuit.measure(q_lh, c_lh)
  circuit.measure(q_wh, c_wh)
  display(circuit.draw())

  simulator = AerSimulator()
  compiled_circuit = transpile(circuit, simulator)
  result = simulator.run(compiled_circuit, shots=1).result()
  counts = result.get_counts()
  print("Counts:", counts)

  # Decode the measured result
  # Add the smallest side along with 2 of each area
  total = 0
  for output in counts:
      results = output.split(' ')
      total += int(min(results), 2)
      for result in results:
        total += 2 * int(result, 2)
  print(f"Final total: {total}")

compute_product(2, 3, 4)

dimension: 010
dimension: 011
dimension: 100


░ ┌───────────────────┐┌───────────────────┐                     »
   l_0: ──────░─┤0                  ├┤0                  ├─────────────────────»
        ┌───┐ ░ │                   ││                   │                     »
   l_1: ┤ X ├─░─┤1                  ├┤1                  ├─────────────────────»
        └───┘ ░ │                   ││                   │                     »
   l_2: ──────░─┤2                  ├┤2                  ├─────────────────────»
        ┌───┐ ░ │                   ││                   │┌───────────────────┐»
   w_0: ┤ X ├─░─┤3                  ├┤                   ├┤0                  ├»
        ├───┤ ░ │                   ││                   ││                   │»
   w_1: ┤ X ├─░─┤4                  ├┤                   ├┤1                  ├»
        └───┘ ░ │                   ││                   ││                   │»
   w_2: ──────░─┤5                  ├┤                   ├┤2                  ├»
              ░ │                   ││                   ││                   │»
   h_0: ──────░─┤                   ├┤3                  ├┤3                  ├»
              ░ │                   ││                   ││                   │»
   h_1: ──────░─┤   RGQFTMultiplier ├┤4                  ├┤4                  ├»
        ┌───┐ ░ │                   ││                   ││                   │»
   h_2: ┤ X ├─░─┤                   ├┤5                  ├┤5                  ├»
        └───┘ ░ │                   ││                   ││                   │»
  lw_0: ──────░─┤6                  ├┤                   ├┤                   ├»
              ░ │                   ││                   ││                   │»
  lw_1: ──────░─┤7                  ├┤   RGQFTMultiplier ├┤                   ├»
              ░ │                   ││                   ││                   │»
  lw_2: ──────░─┤8                  ├┤                   ├┤                   ├»
              ░ │                   ││                   ││                   │»
  lw_3: ──────░─┤9                  ├┤                   ├┤                   ├»
              ░ │                   ││                   ││                   │»
  lw_4: ──────░─┤10                 ├┤                   ├┤                   ├»
              ░ │                   ││                   ││                   │»
  lw_5: ──────░─┤11                 ├┤                   ├┤                   ├»
              ░ └───────────────────┘│                   ││   RGQFTMultiplier │»
  lh_0: ──────░──────────────────────┤6                  ├┤                   ├»
              ░                      │                   ││                   │»
  lh_1: ──────░──────────────────────┤7                  ├┤                   ├»
              ░                      │                   ││                   │»
  lh_2: ──────░──────────────────────┤8                  ├┤                   ├»
              ░                      │                   ││                   │»
  lh_3: ──────░──────────────────────┤9                  ├┤                   ├»
              ░                      │                   ││                   │»
  lh_4: ──────░──────────────────────┤10                 ├┤                   ├»
              ░                      │                   ││                   │»
  lh_5: ──────░──────────────────────┤11                 ├┤                   ├»
              ░                      └───────────────────┘│                   │»
  wh_0: ──────░───────────────────────────────────────────┤6                  ├»
              ░                                           │                   │»
  wh_1: ──────░───────────────────────────────────────────┤7                  ├»
              ░                                           │                   │»
  wh_2: ──────░───────────────────────────────────────────┤8                  ├»
              ░                                           │                   │»
  wh_3: ──────░──────────────────────────────

Counts: {'001100 001000 000110': 1}
Final total: 58
